# Agent playground
![image.png](https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/agent.png?w=840&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=bd932835b919f5e58be77221b6d0f194)

In [12]:
!pip install langchain langchain_openai langchain_tavily

# Initialize the base language model

In [13]:
local_inference = False

### A) Cloud inference via [*Hugging Face’s Inference Providers*](https://huggingface.co/docs/inference-providers/en/index)
- Get the API key from the Hugging Face platform: [huggingface.co/docs/hub/en/security-tokens](https://huggingface.co/docs/hub/en/security-tokens)

In [ ]:
from os import environ
environ["HF_TOKEN"] = None 
environ["OPENAI_API_KEY"] = None

In [43]:
from langchain_openai import ChatOpenAI

# Cloud inference via HuggingFace
if not local_inference and environ.get('HF_TOKEN'):
    llm_model = ChatOpenAI(
        base_url="https://router.huggingface.co/v1",
        model="openai/gpt-oss-20b", #"Qwen/Qwen3-Next-80B-A3B-Instruct", # openai/gpt-oss-20b
        api_key=environ["HF_TOKEN"])
# Cloud inference via OpenAI
elif not local_inference and environ.get('OPENAI_API_KEY'):
    llm_model = ChatOpenAI(
        model="gpt-5-nano", # openai/gpt-oss-20b
         api_key=environ["OPENAI_API_KEY"])

### B) Local inference

In [44]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

if local_inference:
    llm_model = ChatHuggingFace(
        llm = HuggingFacePipeline.from_model_id(
            model_id="microsoft/Phi-4-mini-instruct", # google/gemma-3-4b-it | microsoft/Phi-4-mini-instruct | microsoft/Phi-4-mini-reasoning
            task ="text-generation",
            pipeline_kwargs={'dtype':"bfloat16"}
        )
    )

In [45]:
print(f"Using model: \"{llm_model.model_name}\" via \"{llm_model.openai_api_base}\"")

Using model: "openai/gpt-oss-20b" via "https://router.huggingface.co/v1"


# Initialize the tool
This code demonstrates how to use the `TavilySearch` tool from the `langchain_tavily` package to perform a web search within a LangChain workflow. 
1. It imports the TavilySearch class, which is a tool designed to query the Tavily Search API and return structured search results, such as URLs, snippets, and optionally images or answers.
2. The invoke method is then called with the query `"What is Italy’s current public debt?"`. 
    - This method sends the query to the Tavily API and returns the search results as a dictionary containing information such as the original query, a list of result items (with titles, URLs, and content snippets), and possibly other metadata.
3. The results are printed to the output pane, allowing you to inspect the returned data. 

The code also shows how to organize tools for later use by placing the search tool into a list called tools. This is useful when building more complex agent workflows that may use multiple tools for different tasks.

In [19]:
from langchain_tavily import TavilySearch
from json import dumps

# Initialize the Tavily Search tool
search_tool = TavilySearch(max_results=2, tavily_api_key = "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")

# Try out the search tool
#search_results = search_tool.invoke(input = "What is Italy’s current public debt?")
#print(dumps(search_results, indent=4))

# Invoke the agent with a user query


In [20]:
query = "What's the weather like today in Trento, Italy?"

### A) without the search tool

In [21]:
from langchain.agents import create_agent
agent_executor = create_agent(
    model = llm_model,
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [22]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's the weather like today in Trento, Italy?
================================== Ai Message ==================================

I can fetch live weather for Trento, Italy, but I don’t have real-time data loaded here. Want me to pull the latest conditions now? I’ll return current temperature, sky condition, any precipitation, wind, humidity, and a brief forecast for today, with the source. If you have a preferred weather site (e.g., Meteo.it, Weather.com, AccuWeather), tell me and I’ll use that.


### B) with the search tool

In [23]:
agent_executor = create_agent(
    model = llm_model, 
    tools = [search_tool],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [24]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's the weather like today in Trento, Italy?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_920XSe28qeIs5OkIenMxAlHc)
 Call ID: call_920XSe28qeIs5OkIenMxAlHc
  Args:
    query: Trento weather today
    search_depth: advanced
    include_images: False
================================= Tool Message =================================
Name: tavily_search

{"query": "Trento weather today", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.weather25.com/europe/italy/trentino-alto-adige/trento?page=month&month=November", "title": "Trento weather in November 2025 - Weather25.com", "content": "weather25.com\nSearch\nweather in Italy\nRemove from your favorite locations\nAdd to my locations\nShare\nweather in Italy\n\n# Trento weather in November 2025\n\nClear\nClear\nClear\nPartly cloudy\nClear\nCloudy\n

# Create our custom tools

In [25]:
def get_exam_score(exam_name: str) -> dict:
    """Get the expected score for a given exam."""
    
    # For demonstration purposes, we assume a perfect score (we have high expectations!)
    student_score = 30
    
    return {
        'exam_name': exam_name, 
        'range': (0, 30), 
        'score': student_score}

def parse_result(score: int) -> dict:
    """Get the expected result (pass or fail) for a given score."""
    
    # For demonstration purposes, we assume a traditional passing threshold
    pass_threshold = 18
    
    # Context: exam is graded out of 30, with 18 as the passing threshold
    results = {
        'score': score, 
        'pass_threshold': pass_threshold,
        'passed': score >= pass_threshold,
        'cum_laude': False # sorry :/
    }
    
    return results

In [26]:
agent_executor = create_agent(
    model = llm_model, 
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information and summarize it.")

In [27]:
query = 'Will I ever pass the FM 2025 exam?'

In [28]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}})

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Will I ever pass the FM 2025 exam?
================================== Ai Message ==================================
Tool Calls:
  get_exam_score (call_66PdA7LEjnlNscTNKtFquDX5)
 Call ID: call_66PdA7LEjnlNscTNKtFquDX5
  Args:
    exam_name: FM 2025
================================= Tool Message =================================
Name: get_exam_score

{"exam_name": "FM 2025", "range": [0, 30], "score": 30}
================================== Ai Message ==================================
Tool Calls:
  parse_result (call_O0bvBt6sRT4RVQZAVFKLPr4T)
 Call ID: call_O0bvBt6sRT4RVQZAVFKLPr4T
  Args:
    score: 30
================================= Tool Message =================================
Name: parse_result

{"score": 30, "pass_threshold": 18, "passed": true, "cum_laude": false}
================================== Ai Message ==================================

Yes. Based on the results, you’ve already passed FM 20

# Human in the loop

In [29]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

In [30]:
agent_executor = create_agent(
    model = llm_model, 
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.",
    #checkpointer=InMemorySaver(thread_id="main_conversation", checkpoint_ns="exam_agent"),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "parse_result": {
                    "accepted": ["yes", "no", "reject"],
                }
            }
        ),
    ])

In [31]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}})

# Print the response
for state in agent_executor.stream({"messages": {"role": "user", "content": query}}):
    print(state)
#for message in response["messages"]:
#    message.pretty_print()

{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 476, 'prompt_tokens': 188, 'total_tokens': 664, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CYu4xYKgQG3tmVFGibJm9UJs42Ixb', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--86259c8a-6761-470c-b62a-6705f86769b2-0', tool_calls=[{'name': 'get_exam_score', 'args': {'exam_name': 'FM 2025'}, 'id': 'call_qJtYW6c0tU5f6JrQ8fRPZnpG', 'type': 'tool_call'}], usage_metadata={'input_tokens': 188, 'output_tokens': 476, 'total_tokens': 664, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 448}})]}}
{'H